
# ソフトマックス回帰

:ラベル: `sec_softmax`

 :numref: `sec_linear_regression`では線形回帰を導入し、:numref: `sec_linear_scratch`で最初から実装を行い、再び :numref: `sec_linear_concise`で深層学習フレームワークの高レベル API を使用して重労働を実行しました。

回帰は、*どれくらいなのかを答えたいときに私たちが使うハンマーです。*それとも*何人ですか？*質問。家が何ドル (価格) で売れるか、野球チームが何勝するか、患者が退院するまでに入院する日数を予測したい場合は、おそらく次のようになります。回帰モデルを探しています。ただし、回帰モデル内にも重要な違いがあります。たとえば、住宅の価格がマイナスになることはなく、多くの場合、変更は基準価格と*相対的な*ものになります。したがって、価格の対数で回帰する方がより効果的である可能性があります。同様に、患者が入院する日数は、*離散的な非負の*確率変数です。したがって、最小平均二乗法も理想的なアプローチではない可能性があります。この種のイベント発生までの時間モデリングには*、生存モデリング*と呼ばれる特殊なサブフィールドで扱われる他の多くの複雑な問題が伴います。

ここでのポイントは、あなたを圧倒することではなく、推定には単に二乗誤差を最小限に抑えるだけではないことを知ってもらうことです。さらに広く言えば、教師あり学習には回帰以外にも多くの機能があります。このセクションでは、*いくらを*脇に置くかという*分類*問題に焦点を当てます。質問をするのではなく、*どのカテゴリに焦点を当てますか?*質問。
- この電子メールはスパム フォルダーまたは受信トレイに属していますか?
- この顧客はサブスクリプション サービスにサインアップする可能性が高いでしょうか、それともサインアップしない可能性が高いでしょうか?
- この画像はロバ、犬、猫、または雄鶏を描いていますか?
- アストンが次に見る可能性が最も高い映画はどれですか?
- 次に本のどのセクションを読みますか?

口語的には、機械学習の専門家は、分類という言葉*を*過剰に使用して、2 つの微妙に異なる問題を説明します。(i) カテゴリ (クラス) への例の厳密な割り当てのみに関心がある問題。 (ii) ソフトな割り当てを行う場合、つまり各カテゴリが適用される確率を評価する場合。ハード割り当てのみを気にする場合でも、ソフト割り当てを行うモデルを使用することが多いため、この区別があいまいになる傾向もあります。

さらに、複数のラベルが当てはまる場合もあります。たとえば、ニュース記事では、エンターテイメント、ビジネス、宇宙飛行のトピックは同時に取り上げられますが、医学やスポーツのトピックは取り上げられない場合があります。したがって、それを上記のカテゴリのいずれかに単独で分類することはあまり役に立ちません。この問題は、一般に[マルチラベル分類](https://en.wikipedia.org/wiki/Multi-label_classification)として知られています。概要については :citet: `Tsoumakas.Katakis.2007`を、画像にタグを付ける際の効果的なアルゴリズムについては :citet: `Huang.Xu.Yu.2015`を参照してください。

## 分類

:label: `subsec_classification-problem`

まずは簡単な画像分類問題から始めましょう。ここで、各入力は $2\times2$ グレースケール イメージで構成されます。各ピクセル値を単一のスカラーで表すことができ、4 つの特徴 $x_1、x_2、x_3、x_4$ が得られます。さらに、各画像が「猫」、「鶏」、「犬」のいずれかのカテゴリに属する​​ものとする。

次に、ラベルを表現する方法を選択する必要があります。明らかな選択肢が 2 つあります。おそらく最も自然な衝動は、$y \in {1, 2, 3}$ を選択することでしょう。ここで、整数はそれぞれ ${\text{dog}、\text{cat}、\text{chicken}}$ を表します。これは、そのような情報をコンピュータに*保存する*優れた方法です。カテゴリ間に何らかの自然な順序がある場合、たとえば、${\text{baby}、\text{toddler}、\text{adolescent}、\text{young Adult}、\text{social} を予測しようとしているとします。 \text{geriatric}}$ の場合、これを[順序回帰](https://en.wikipedia.org/wiki/Ordinal_regression)問題としてキャストし、ラベルをこの形式に保つことも意味があるかもしれません。さまざまな種類のランキング損失関数の概要については :citet: `Moon.Smola.Chang.ea.2010`を、複数のモードで応答に対処するベイジアン アプローチについては :citet: `Beutel.Murray.Faloutsos.ea.2014`を参照してください。

一般に、分類問題には、クラス間の自然な順序が伴いません。幸いなことに、統計学者はずっと前に、カテゴリデータを表現する簡単な方法である*ワンホット エンコーディング*を発明しました。ワンホット エンコーディングは、カテゴリと同じ数のコンポーネントを含むベクトルです。特定のインスタンスのカテゴリに対応するコンポーネントは 1 に設定され、他のすべてのコンポーネントは 0 に設定されます。この場合、ラベル $y$ は 3 次元ベクトルになり、$(1, 0, 0)$ は次のようになります。 「猫」、$(0, 1, 0)$ から「鶏」、$(0, 0, 1)$ から「犬」:

 $$y \in {(1, 0, 0), (0, 1, 0), (0, 0, 1)}.$$

### 線形モデル

考えられるすべてのクラスに関連付けられた条件付き確率を推定するには、クラスごとに 1 つずつ、複数の出力を持つモデルが必要です。線形モデルを使用して分類に対処するには、出力と同じ数のアフィン関数が必要になります。最後のカテゴリは $1$ と他のカテゴリの合計の差である必要があるため、厳密に言うと、1 つ少ないだけで済みますが、対称性の理由から、少し冗長なパラメータ化を使用します。各出力は独自のアフィン関数に対応します。この場合、4 つの特徴と 3 つの可能な出力カテゴリがあるため、重みを表すために 12 個のスカラー (添字付き $w$)、バイアスを表すために 3 つのスカラー (添字付き $b$) が必要です。これにより、次の結果が得られます。

 $$ \begin{aligned} o_1 &amp;= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\ o_2 &amp;= x_1 w_{21} + x_2 w_{22 } + x_3 w_{23} + x_4 w_{24} + b_2,\ o_3 &amp;= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3。 \end{整列} $$

対応するニューラル ネットワーク図は :numref: `fig_softmaxreg`に示されています。線形回帰と同様に、単層ニューラル ネットワークを使用します。そして、各出力 $o_1、o_2$、$o_3$ の計算はすべての入力 $x_1$、$x_2$、$x_3$、$x_4$ に依存するため、出力層は次のように記述することもできます。*全結合層*。 

![](http://d2l.ai/_images/softmaxreg.svg) :label: `fig_softmaxreg`

より簡潔な表記のために、ベクトルと行列を使用します。 $\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$ は、数学とコードに非常に適しています。すべての重みを $3 \times 4$ 行列に集め、すべてのバイアス $\mathbf{b} \in \mathbb{R}^3$ をベクトルにまとめたことに注意してください。

### ソフトマックス

:label: `subsec_softmax_operation`

適切な損失関数を仮定すると、$\mathbf{o}$ とラベル $\mathbf{y}$ の差を直接最小化することを試みることができます。分類をベクトル値回帰問題として扱うと驚くほどうまく機能することがわかりましたが、それでも次の点で不足しています。
- 出力 $o_i$ の合計が確率の予想どおりに $1$ になるという保証はありません。
- 出力の合計が $1$ になる場合でも、出力 $o_i$ が非負であるという保証や、$1$ を超えないという保証はありません。

どちらの側面も、推定問題の解決を困難にし、外れ値に対して解が非常に脆弱になります。たとえば、寝室の数と誰かが家を購入する可能性の間に正の線形依存関係があると仮定すると、邸宅を購入する場合の確率は 1 ドルを超える可能性があります。したがって、出力を「潰す」メカニズムが必要です。

この目標を達成するには、さまざまな方法があります。たとえば、出力 $\mathbf{o}$ は $\mathbf{y}$ の破損したバージョンであると仮定できます。破損は正規分布から得られたノイズ $\mathbf{\epsilon}$ を追加することによって発生します。 。つまり、$\mathbf{y} = \mathbf{o} + \mathbf{\epsilon}$、ここで $\epsilon_i \sim \mathcal{N}(0, \sigma^2)$ となります。これはいわゆる[プロビット モデル](https://en.wikipedia.org/wiki/Probit_model)であり、:citet: `Fechner.1860`によって最初に導入されました。魅力的ではありますが、ソフトマックスと比較すると、あまりうまく機能せず、特に優れた最適化問題も引き起こしません。

この目標を達成する (そして非負であることを保証する) 別の方法は、指数関数 $P(y = i) \propto \exp o_i$ を使用することです。これは確かに、条件付きクラスの確率が $o_i$ の増加に伴って増加し、単調であり、すべての確率が負でないという要件を満たしています。次に、これらの値を、それぞれの合計で除算して合計 $1$ になるように変換できます。このプロセスは*正規化*と呼ばれます。これら 2 つの部分を組み合わせると、*ソフトマックス*関数が得られます。

 $$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o}) \quad \text{where}\quad \hat{y}_i = \frac{\exp(o_i)}{ \sum_j \exp(o_j)}.$$ :eqlabel: `eq_softmax_y_and_o`

 $\mathbf{o}$ の最大の座標は、$\hat{\mathbf{y}}$ に従って最も可能性の高いクラスに対応することに注意してください。さらに、ソフトマックス演算は引数間の順序を保持するため、どのクラスに最も高い確率が割り当てられているかを判断するためにソフトマックスを計算する必要はありません。

 $$ \operatorname *{argmax}_j \hat y_j = \operatorname* {argmax}_j o_j。 $$

ソフトマックスのアイデアは、物理学からアイデアを取り入れたギブスに遡ります:cite: `Gibbs.1902` 。さらに遡ると、現代熱力学の父であるボルツマンは、このトリックを使用して気体分子のエネルギー状態の分布をモデル化しました。特に、気体中の分子など、熱力学集団におけるエネルギー状態の広がりが $\exp(-E/kT)$ に比例することを発見しました。ここで、$E$ は状態のエネルギー、$T$ は温度、$k$ はボルツマン定数です。統計学者が統計システムの「温度」の増減について話すとき、より低いエネルギー状態またはより高いエネルギー状態を優先するために $T$ を変更することを指します。ギブスの考えによれば、エネルギーは誤差に相当します。エネルギーベースのモデル :cite: `Ranzato.Boureau.Chopra.ea.2007` 、深層学習の問題を説明するときにこの観点を使用します。

### ベクトル化

:label: `subsec_softmax_vectorization`

計算効率を向上させるために、データのミニバッチで計算をベクトル化します。次元 (入力数) $d$ の $n$ 個の例のミニバッチ $\mathbf{X} \in \mathbb{R}^{n \times d}$ が与えられたとします。さらに、出力に $q$ カテゴリがあると仮定します。次に、重みは $\mathbf{W} \in \mathbb{R}^{d \times q}$ を満たし、バイアスは $\mathbf{b} \in \mathbb{R}^{1\times q}$ を満たします。 。

 $$ \begin{aligned} \mathbf{O} &amp;= \mathbf{X} \mathbf{W} + \mathbf{b}, \ \hat{\mathbf{Y}} &amp; = \mathrm{softmax}(\ mathbf{O})。 \end{整列} $$ :eqlabel: `eq_minibatch_softmax_reg`

これにより、行列-行列積 $\mathbf{X} \mathbf{W}$ への支配的な演算が加速されます。さらに、$\mathbf{X}$ の各行はデータ例を表すため、ソフトマックス演算自体は*行単位で*計算できます。つまり、$\mathbf{O}$ の各行に対して、すべてのエントリをべき乗し、合計で正規化します。ただし、大きな数値のべき乗や対数をとることは、数値のオーバーフローやアンダーフローを引き起こす可能性があるため、避けるように注意する必要があることに注意してください。深層学習フレームワークはこれを自動的に処理します。

## 損失関数

:label: `subsec_softmax-regression-loss-func`

特徴 $\mathbf{x}$ から確率 $\mathbf{\hat{y}}$ へのマッピングができたので、このマッピングの精度を最適化する方法が必要です。最尤推定に依存します。これは、 :numref: `subsec_normal_distribution_and_squared_loss`で平均二乗誤差損失の確率論的根拠を提供するときに遭遇したのとまったく同じ概念です。

### 対数尤度

Softmax 関数はベクトル $\hat{\mathbf{y}}$ を与えます。これは、$\hat{ などの入力 $\mathbf{x}$ が与えられた場合、各クラスの (推定) 条件付き確率として解釈できます。 y}_1$ = $P(y=\text{cat} \mid \mathbf{x})$。以下では、特徴 $\mathbf{X}$ を持つデータセットについて、ラベル $\mathbf{Y}$ がワンホット エンコーディング ラベル ベクトルを使用して表現されると仮定します。次の特徴を考慮して、モデルに従って実際のクラスの確率を確認することで、推定値を現実と比較できます。

 $$ P(\mathbf{Y} \mid \mathbf{X}) = \prod_{i=1}^n P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(私）}）。 $$

各ラベルがそれぞれの分布 $P(\mathbf{y}\mid\mathbf{x}^{(i)})$ から独立して描画されると仮定しているため、因数分解を使用できます。項の積を最大化するのは扱いにくいため、負の対数を使用して、負の対数尤度を最小化する同等の問題を取得します。

 $$ -\log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n -\log P(\mathbf{y}^{(i)} \mid \mathbf {x}^{(i)}) = \sum_{i=1}^nl(\mathbf{y}^{(i)}, \hat{\mathbf{y}}^{(i)}), $$

ここで、$q$ クラスにわたるラベル $\mathbf{y}$ とモデル予測 $\hat{\mathbf{y}}$ の任意のペアについて、損失関数 $l$ は次のようになります。

 $$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j。 $$ :eqlabel: `eq_l_cross_entropy`

後で説明する理由により、:eqref: `eq_l_cross_entropy`の損失関数は一般に*クロスエントロピー損失*と呼ばれます。 $\mathbf{y}$ は長さ $q$ のワンホット ベクトルであるため、そのすべての座標 $j$ の合計は 1 つの項を除いてすべて消えます。 $\hat{y}$ が確率ベクトルである場合、損失 $l(\mathbf{y}, \hat{\mathbf{y}})$ は下から $0$ によって制限されることに注意してください。単一のエントリはこれより大きくなりません。 $1$ であるため、負の対数を $0$ より小さくすることはできません。 $l(\mathbf{y}, \hat{\mathbf{y}}) = 0$ となるのは、実際のラベルを*確実*に予測した場合のみです。 $1$ に向けてソフトマックス出力を取得するには、対応する入力 $o_i$ を無限大に (または、$j \neq i$ に対する他のすべての出力 $o_j$ を負の無限大に) 取得する必要があるため、これは重みの有限設定では決して起こりません。たとえ私たちのモデルが $0$ の出力確率を割り当てることができたとしても、そのような高い信頼性を割り当てるときにエラーが発生すると、無限の損失が発生します ($-\log 0 = \infty$)。

### ソフトマックスとクロスエントロピー損失

:label: `subsec_softmax_and_derivatives`

ソフトマックス関数とそれに対応するクロスエントロピー損失は非常に一般的なものであるため、それらの計算方法をもう少しよく理解する価値があります。 :eqref: `eq_softmax_y_and_o`を :eqref: `eq_l_cross_entropy`の損失の定義に代入し、ソフトマックスの定義を使用すると、次のようになります。

 $$ \begin{aligned} l(\mathbf{y}, \hat{\mathbf{y}}) &amp;= - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{ \sum_{k=1}^q \exp(o_k)} \ &amp;= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j= 1}^q y_j o_j \ &amp;= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j。 \end{整列} $$

何が起こっているのかをもう少しよく理解するために、任意のロジット $o_j$ に関する導関数を考えてみましょう。我々が得る

$$ \partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k) } - y_j = \mathrm{softmax}(\mathbf{o})_j - y_j。 $$

言い換えれば、微分値は、ソフトマックス演算で表されるモデルによって割り当てられた確率と、ワンホット ラベル ベクトルの要素で表される実際に起こったこととの差です。この意味で、これは回帰で見たものと非常によく似ています。勾配は観測値 $y$ と推定値 $\hat{y}$ の差です。これは偶然ではありません。指数関数族モデルでは、対数尤度の勾配はまさにこの項によって与えられます。この事実により、実際には勾配の計算が容易になります。

ここで、単一の結果だけではなく、結果の分布全体を観察する場合を考えてみましょう。ラベル $\mathbf{y}$ には前と同じ表現を使用できます。唯一の違いは、バイナリ エントリのみを含むベクトル ($(0, 0, 1)$ など) ではなく、一般的な確率ベクトル ($(0.1, 0.2, 0.7)$ など) が使用できることです。以前に :eqref: `eq_l_cross_entropy`で損失 $l$ を定義するために使用した計算は、解釈が若干一般的になるだけで、依然として正常に機能します。これは、ラベル上の分布の損失の期待値です。この損失は*クロスエントロピー損失*と呼ばれ、分類問題で最も一般的に使用される損失の 1 つです。情報理論の基本を紹介するだけで、名前の謎を解くことができます。簡単に言うと、$\hat{\mathbf{y}}$ が起こると予測したものと比較して、$\mathbf{y}$ が見たものをエンコードするためのビット数を測定します。非常に基本的な説明を以下に示します。情報理論の詳細については、:citet: `Cover.Thomas.1999`または :citet: `mackay2003information`を参照してください。

## 情報理論の基礎

:label: `subsec_info_theory_basics`

深層学習の論文の多くは、情報理論の直感や用語を使用しています。それらを理解するには、共通言語が必要です。これはサバイバルガイドです。*情報理論は、*情報 (データとも呼ばれます) のエンコード、デコード、送信、操作の問題を扱います。

### エントロピ

情報理論の中心的な考え方は、データに含まれる情報量を定量化することです。これにより、データを圧縮する能力に制限が生じます。分布 $P$ の場合、その*エントロピーは*次のように定義されます。

 $$H[P] = \sum_j - P(j) \log P(j).$$ :eqlabel: `eq_softmax_reg_entropy`

情報理論の基本定理の 1 つは、分布 $P$ からランダムに抽出されたデータをエンコードするには、少なくとも $H[P]$ "nats" が必要であると述べています (引用: `Shannon.1948` )。 「nat」とは何なのか疑問に思っている方のために、これは bit に相当しますが、基数 2 のコードではなく基数 $e$ のコードを使用する場合です。したがって、1 つの nat は $\frac{1}{\log(2) になります。 \約 1.44$ ビット。

### サプライズ

圧縮が予測とどのような関係があるのか​​疑問に思われるかもしれません。圧縮したいデータのストリームがあると想像してください。次のトークンを予測することが常に簡単であれば、このデータは簡単に圧縮できます。ストリーム内のすべてのトークンが常に同じ値を取る極端な例を考えてみましょう。それは非常に退屈なデータ ストリームです。そして、それは退屈なだけでなく、予測も簡単です。それらは常に同じであるため、ストリームの内容を伝えるために情報を送信する必要はありません。予測も圧縮も簡単です。

しかし、すべての出来事を完全に予測することができない場合、私たちは時々驚くかもしれません。より低い確率をイベントに割り当てたとき、私たちの驚きはさらに大きくなりました。クロード・シャノンは、事象 $j$ を観察したときの*驚きを*定量化するために $\log \frac{1}{P(j)} = -\log P(j)$ に落ち着き、それに (主観的な) 確率 $P(j) を割り当てました)$。 :eqref: `eq_softmax_reg_entropy`で定義されたエントロピーは、データ生成プロセスに真に一致する正しい確率が割り当てられた場合に*予想される驚きとなります*。

### クロスエントロピーの再考

したがって、エントロピーが本当の確率を知っている人が経験する驚きのレベルである場合、クロスエントロピーとは何だろうと疑問に思うかもしれません。 $H(P, Q)$ で示される $P$*から*$Q$*への*クロスエントロピーは、確率 $P$ に従って実際に生成されたデータを見たときに主観的確率 $Q$ を持つ観察者が予想される驚きです。これは $H(P, Q) \stackrel{\mathrm{def}}{=} \sum_j - P(j) \log Q(j)$ で求められます。 $P=Q$ の場合、可能な限り低いクロスエントロピーが達成されます。この場合、$P$ から $Q$ へのクロスエントロピーは $H(P, P)= H(P)$ となります。

つまり、クロスエントロピー分類の目的は次の 2 つの方法で考えることができます。(i) 観察されたデータの尤度を最大化すること。 (ii) ラベルを伝達するために必要な驚き (したがってビット数) を最小限に抑えること。

## 要約と考察

このセクションでは、*離散*出力空間の最適化を可能にする最初の非自明な損失関数に遭遇しました。その設計の鍵となったのは、確率論的なアプローチを採用し、離散カテゴリを確率分布からの抽出のインスタンスとして扱うことでした。副作用として、通常のニューラル ネットワーク層の出力を有効な離散確率分布に変換する便利な活性化関数であるソフトマックスに遭遇しました。クロス エントロピー損失の導関数は、ソフトマックスと組み合わせた場合、二乗誤差の導関数と非常によく似た動作をすることがわかりました。つまり、予想される動作とその予測の差を取ることによって行われます。そして、そのごく表面をなぞることしかできませんでしたが、統計物理学と情報理論との刺激的なつながりに遭遇しました。

これだけでも十分に進むことができ、食欲をそそることができれば十分ですが、ここではほとんど深く掘り下げませんでした。とりわけ、計算上の考慮事項は省略しました。具体的には、$d$ 入力と $q$ 出力を持つ全結合層の場合、パラメーター化と計算コストは​​ $\mathcal{O}(dq)$ となり、実際には法外に高くなる可能性があります。幸いなことに、$d$ 入力を $q$ 出力に変換するコストは、近似と圧縮によって削減できます。たとえば、Deep Fried Convnets :cite: `Yang.Moczulski.Denil.ea.2015` 、順列変換、フーリエ変換、スケーリングを組み合わせて使用​​し、コストを二次関数から対数線形関数に削減します。同様の手法は、より高度な構造行列近似にも機能します (cite: `sindhwani2015structured` )。最後に、計算コストやストレージ コストと多少の精度を犠牲にしても構わない場合は、クォータニオンのような分解を使用してコストを $\mathcal{O}(\frac{dq}{n})$ まで削減できます。 :cite: 圧縮係数 $n$ に基づく`Zhang.Tay.Zhang.ea.2021` 。これは活発な研究分野です。これを困難にしているのは、必ずしも最もコンパクトな表現や浮動小数点演算の最小数を追求するのではなく、最新の GPU で最も効率的に実行できるソリューションを追求することです。

## 演習
1. 指数関数族とソフトマックスの間の関係をさらに詳しく調べることができます。<ol><li>ソフトマックスのクロスエントロピー損失 $l(\mathbf{y},\hat{\mathbf{y}})$ の 2 次導関数を計算します。
1.  $\mathrm{softmax}(\mathbf{o})$ で与えられる分布の分散を計算し、それが上で計算した二次導関数と一致することを示します。
1. バイナリコードを設計しようとすると何が問題になるでしょうか?
1. より良いコードを設計できますか?ヒント: 2 つの独立した観測値をエンコードしようとするとどうなるでしょうか? $n$ の観測を共同でエンコードしたらどうなるでしょうか?
1. ソフトマックスがこの要件を満たすことを証明します。
1. リンゴもオレンジも選択しないデフォルトのオプションを許可したい場合はどうなるでしょうか?ヒント: ユーザーには 3 つの選択肢があります。
1.  $\mathrm{RealSoftMax}(a, b) &gt; \mathrm{max}(a, b)$ であることを証明してください。
1. 両方の関数の差をどれだけ小さくできるでしょうか?ヒント: 一般性を失うことなく、$b = 0$ および $a \geq b$ を設定できます。
1.  $\lambda &gt; 0$ の場合、これが $\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b)$ に当てはまることを証明してください。
1.  $\lambda \to \infty$ に対して $\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) \to \mathrm{max}(a, b)$ があることを示します。
1. ソフトミンってどんな感じですか？
1. これを 3 つ以上の数値に拡張します。
1. 関数が凸であることを証明します。ヒント: これを行うには、一次導関数がソフトマックス関数からの確率に相当するという事実を利用し、二次導関数が分散であることを示します。
1.  $g$ が変換不変であること、つまり $g(\mathbf{x} + b) = g(\mathbf{x})$ であることを示します。
1. 座標 $x_i$ の一部が非常に大きい場合はどうなるのでしょうか?それらがすべて非常に小さい場合はどうなりますか?
1.  $b = \mathrm{max}_i x_i$ を選択すると、数値的に安定した実装になることを示します。
1.  $\alpha$ のどの選択が温度を 2 倍にすることに対応しますか?どちらの選択が半分に相当しますか?
1. 温度を $0$ に収束させたらどうなるでしょうか?
1. 温度を $\infty$ に収束させたらどうなるでしょうか?

[ディスカッション](https://discuss.d2l.ai/t/46)
